In [1]:
from __future__ import print_function, division

from keras.datasets import mnist,fashion_mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import losses
from keras.utils import to_categorical
import keras.backend as K

import matplotlib.pyplot as plt
import os
import numpy as np

class SGAN:
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(
            loss=['binary_crossentropy', 'categorical_crossentropy'],
            loss_weights=[0.5, 0.5],
            optimizer=optimizer,
            metrics=['accuracy']
        )

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        noise = Input(shape=(100,))
        img = self.generator(noise)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid, _ = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model(noise, valid)
        self.combined.compile(loss=['binary_crossentropy'], optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(1, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())

        model.summary()

        img = Input(shape=self.img_shape)

        features = model(img)
        valid = Dense(1, activation="sigmoid")(features)
        label = Dense(self.num_classes+1, activation="softmax")(features)

        return Model(img, [valid, label])

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (_, _) = fashion_mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Class weights:
        # To balance the difference in occurences of digit class labels.
        # 50% of labels that the discriminator trains on are 'fake'.
        # Weight = 1 / frequency
        half_batch = batch_size // 2
        cw1 = {0: 1, 1: 1}
        cw2 = {i: self.num_classes / half_batch for i in range(self.num_classes)}
        cw2[self.num_classes] = 1 / half_batch

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                labels = to_categorical(y_train[index * batch_size:(index + 1) * batch_size], num_classes=self.num_classes + 1)
                # Sample noise and generate a batch of new images
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                gen_imgs = self.generator.predict(noise)

                # One-hot encoding of labels
                # labels = to_categorical(y_train[idx], num_classes=self.num_classes + 1)
                fake_labels = to_categorical(np.full((batch_size, 1), self.num_classes),
                                             num_classes=self.num_classes + 1)

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(imgs, [valid, labels], class_weight=[cw1, cw2])
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, [fake, fake_labels], class_weight=[cw1, cw2])
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                g_loss = self.combined.train_on_batch(noise, valid, class_weight=[cw1, cw2])

                # Plot the progress
                print("epoch:%d step:%d[D loss: %f, acc: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (
                    epoch, global_step, d_loss[0], 100 * d_loss[3], 100 * d_loss[4], g_loss))

                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    self.sample_images(epoch, global_step)

    def sample_images(self, epoch, global_step):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
                axs[i, j].axis('off')
                cnt += 1
        if not os.path.isdir('images_sgan_fashion_mnist'):
            os.mkdir('images_sgan_fashion_mnist')
        fig.savefig("images_sgan_fashion_mnist/epoch_%d_step_%d.png" % (epoch, global_step))
        plt.close()

if __name__ == '__main__':
    sgan = SGAN()
    sgan.train(epochs=40, batch_size=64, sample_interval=200)



Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 64)          0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1[D loss: 0.656028, acc: 40.62%, op_acc: 7.81%] [G loss: 0.914944]
epoch:0 step:2[D loss: 0.554355, acc: 63.28%, op_acc: 12.50%] [G loss: 0.990527]
epoch:0 step:3[D loss: 0.466609, acc: 75.00%, op_acc: 16.41%] [G loss: 1.044550]
epoch:0 step:4[D loss: 0.445543, acc: 82.03%, op_acc: 19.53%] [G loss: 1.355230]
epoch:0 step:5[D loss: 0.437562, acc: 77.34%, op_acc: 17.97%] [G loss: 1.260653]
epoch:0 step:6[D loss: 0.426240, acc: 78.12%, op_acc: 18.75%] [G loss: 1.348240]
epoch:0 step:7[D loss: 0.369986, acc: 85.16%, op_acc: 28.91%] [G loss: 1.481423]
epoch:0 step:8[D loss: 0.421468, acc: 77.34%, op_acc: 26.56%] [G loss: 1.310434]
epoch:0 step:9[D loss: 0.409896, acc: 76.56%, op_acc: 21.09%] [G loss: 1.387135]
epoch:0 step:10[D loss: 0.436348, acc: 71.88%, op_acc: 26.56%] [G loss: 1.546521]
epoch:0 step:11[D loss: 0.439076, acc: 71.88%, op_acc: 28.12%] [G loss: 1.470062]
epoch:0 step:12[D loss: 0.369971, acc: 80.47%, op_acc: 33.59%] [G loss: 1.599443]
epoch:0 step:13[D loss: 0.

epoch:0 step:102[D loss: 0.470238, acc: 62.50%, op_acc: 50.78%] [G loss: 1.384715]
epoch:0 step:103[D loss: 0.492220, acc: 56.25%, op_acc: 51.56%] [G loss: 1.436571]
epoch:0 step:104[D loss: 0.374166, acc: 68.75%, op_acc: 53.91%] [G loss: 1.971096]
epoch:0 step:105[D loss: 0.402836, acc: 67.97%, op_acc: 50.78%] [G loss: 1.804822]
epoch:0 step:106[D loss: 0.318537, acc: 77.34%, op_acc: 50.78%] [G loss: 1.613841]
epoch:0 step:107[D loss: 0.350748, acc: 67.97%, op_acc: 62.50%] [G loss: 1.640213]
epoch:0 step:108[D loss: 0.391206, acc: 67.97%, op_acc: 54.69%] [G loss: 1.567173]
epoch:0 step:109[D loss: 0.393707, acc: 61.72%, op_acc: 56.25%] [G loss: 1.937393]
epoch:0 step:110[D loss: 0.333030, acc: 74.22%, op_acc: 56.25%] [G loss: 1.804954]
epoch:0 step:111[D loss: 0.421455, acc: 63.28%, op_acc: 52.34%] [G loss: 1.396618]
epoch:0 step:112[D loss: 0.354138, acc: 68.75%, op_acc: 61.72%] [G loss: 1.436782]
epoch:0 step:113[D loss: 0.363499, acc: 71.09%, op_acc: 57.81%] [G loss: 1.237061]
epoc

...

epoch:39 step:37015[D loss: 0.368815, acc: 65.62%, op_acc: 48.44%] [G loss: 1.450626]
epoch:39 step:37016[D loss: 0.241326, acc: 88.28%, op_acc: 55.47%] [G loss: 1.976574]
epoch:39 step:37017[D loss: 0.267100, acc: 78.91%, op_acc: 55.47%] [G loss: 1.819694]
epoch:39 step:37018[D loss: 0.155263, acc: 92.97%, op_acc: 74.22%] [G loss: 2.223431]
epoch:39 step:37019[D loss: 0.381438, acc: 60.16%, op_acc: 51.56%] [G loss: 1.329305]
epoch:39 step:37020[D loss: 0.345552, acc: 60.94%, op_acc: 60.94%] [G loss: 1.476757]
epoch:39 step:37021[D loss: 0.413045, acc: 58.59%, op_acc: 53.12%] [G loss: 1.781022]
epoch:39 step:37022[D loss: 0.300429, acc: 78.91%, op_acc: 57.81%] [G loss: 1.883313]
epoch:39 step:37023[D loss: 0.299723, acc: 75.00%, op_acc: 67.97%] [G loss: 1.914068]
epoch:39 step:37024[D loss: 0.349459, acc: 64.84%, op_acc: 58.59%] [G loss: 1.366971]
epoch:39 step:37025[D loss: 0.244913, acc: 84.38%, op_acc: 54.69%] [G loss: 1.828235]
epoch:39 step:37026[D loss: 0.312925, acc: 77.34%, op_

epoch:39 step:37115[D loss: 0.186861, acc: 88.28%, op_acc: 66.41%] [G loss: 1.297846]
epoch:39 step:37116[D loss: 0.322502, acc: 70.31%, op_acc: 57.03%] [G loss: 1.669310]
epoch:39 step:37117[D loss: 0.358963, acc: 62.50%, op_acc: 57.03%] [G loss: 1.501020]
epoch:39 step:37118[D loss: 0.373284, acc: 63.28%, op_acc: 57.81%] [G loss: 1.343097]
epoch:39 step:37119[D loss: 0.238254, acc: 84.38%, op_acc: 66.41%] [G loss: 1.532701]
epoch:39 step:37120[D loss: 0.312516, acc: 75.00%, op_acc: 60.94%] [G loss: 1.457526]
epoch:39 step:37121[D loss: 0.329444, acc: 64.06%, op_acc: 57.03%] [G loss: 1.186258]
epoch:39 step:37122[D loss: 0.368909, acc: 69.53%, op_acc: 52.34%] [G loss: 1.429696]
epoch:39 step:37123[D loss: 0.349657, acc: 72.66%, op_acc: 58.59%] [G loss: 1.568557]
epoch:39 step:37124[D loss: 0.308321, acc: 67.97%, op_acc: 60.16%] [G loss: 1.705987]
epoch:39 step:37125[D loss: 0.427080, acc: 55.47%, op_acc: 53.12%] [G loss: 1.698519]
epoch:39 step:37126[D loss: 0.267492, acc: 82.03%, op_

epoch:39 step:37211[D loss: 0.348798, acc: 69.53%, op_acc: 61.72%] [G loss: 1.696089]
epoch:39 step:37212[D loss: 0.200947, acc: 89.84%, op_acc: 64.84%] [G loss: 1.484778]
epoch:39 step:37213[D loss: 0.369178, acc: 65.62%, op_acc: 56.25%] [G loss: 1.602936]
epoch:39 step:37214[D loss: 0.299176, acc: 77.34%, op_acc: 56.25%] [G loss: 1.433856]
epoch:39 step:37215[D loss: 0.262405, acc: 77.34%, op_acc: 59.38%] [G loss: 1.368960]
epoch:39 step:37216[D loss: 0.346181, acc: 68.75%, op_acc: 53.12%] [G loss: 2.093213]
epoch:39 step:37217[D loss: 0.279943, acc: 80.47%, op_acc: 57.81%] [G loss: 1.746828]
epoch:39 step:37218[D loss: 0.311217, acc: 70.31%, op_acc: 67.19%] [G loss: 1.969951]
epoch:39 step:37219[D loss: 0.264208, acc: 78.91%, op_acc: 57.81%] [G loss: 1.452070]
epoch:39 step:37220[D loss: 0.294568, acc: 79.69%, op_acc: 59.38%] [G loss: 1.840861]
epoch:39 step:37221[D loss: 0.400381, acc: 58.59%, op_acc: 57.81%] [G loss: 1.329419]
epoch:39 step:37222[D loss: 0.276060, acc: 75.78%, op_

epoch:39 step:37310[D loss: 0.299966, acc: 77.34%, op_acc: 51.56%] [G loss: 1.382087]
epoch:39 step:37311[D loss: 0.242951, acc: 78.12%, op_acc: 63.28%] [G loss: 1.753111]
epoch:39 step:37312[D loss: 0.279982, acc: 76.56%, op_acc: 60.94%] [G loss: 1.106996]
epoch:39 step:37313[D loss: 0.402760, acc: 57.03%, op_acc: 53.91%] [G loss: 1.241114]
epoch:39 step:37314[D loss: 0.261718, acc: 78.12%, op_acc: 59.38%] [G loss: 1.402137]
epoch:39 step:37315[D loss: 0.335592, acc: 67.97%, op_acc: 57.03%] [G loss: 1.302624]
epoch:39 step:37316[D loss: 0.326526, acc: 72.66%, op_acc: 55.47%] [G loss: 1.332727]
epoch:39 step:37317[D loss: 0.327995, acc: 71.09%, op_acc: 53.12%] [G loss: 1.565825]
epoch:39 step:37318[D loss: 0.335571, acc: 69.53%, op_acc: 57.03%] [G loss: 1.401647]
epoch:39 step:37319[D loss: 0.241978, acc: 82.03%, op_acc: 55.47%] [G loss: 1.616888]
epoch:39 step:37320[D loss: 0.371079, acc: 65.62%, op_acc: 57.81%] [G loss: 1.863409]
epoch:39 step:37321[D loss: 0.250479, acc: 82.03%, op_

epoch:39 step:37406[D loss: 0.239417, acc: 83.59%, op_acc: 62.50%] [G loss: 1.998393]
epoch:39 step:37407[D loss: 0.327387, acc: 69.53%, op_acc: 51.56%] [G loss: 1.820771]
epoch:39 step:37408[D loss: 0.357208, acc: 71.09%, op_acc: 54.69%] [G loss: 2.616752]
epoch:39 step:37409[D loss: 0.282738, acc: 75.00%, op_acc: 63.28%] [G loss: 1.695758]
epoch:39 step:37410[D loss: 0.443441, acc: 52.34%, op_acc: 48.44%] [G loss: 1.173490]
epoch:39 step:37411[D loss: 0.220166, acc: 84.38%, op_acc: 60.94%] [G loss: 1.668525]
epoch:39 step:37412[D loss: 0.238033, acc: 79.69%, op_acc: 56.25%] [G loss: 1.704423]
epoch:39 step:37413[D loss: 0.385146, acc: 64.84%, op_acc: 55.47%] [G loss: 1.403243]
epoch:39 step:37414[D loss: 0.221565, acc: 85.94%, op_acc: 62.50%] [G loss: 1.202594]
epoch:39 step:37415[D loss: 0.305048, acc: 75.00%, op_acc: 62.50%] [G loss: 1.533973]
epoch:39 step:37416[D loss: 0.241840, acc: 81.25%, op_acc: 63.28%] [G loss: 1.355887]
epoch:39 step:37417[D loss: 0.191188, acc: 86.72%, op_